In [1]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional, Union
from tqdm import tqdm

import datasets
import numpy as np
import torch
from datasets import load_dataset

'''
import transformers
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
'''

'\nimport transformers\nfrom transformers import (\n    AutoConfig,\n    AutoModelForMultipleChoice,\n    AutoTokenizer,\n    HfArgumentParser,\n    Trainer,\n    TrainingArguments,\n    default_data_collator,\n    set_seed,\n)\nfrom transformers.file_utils import PaddingStrategy\nfrom transformers.tokenization_utils_base import PreTrainedTokenizerBase\nfrom transformers.trainer_utils import get_last_checkpoint\nfrom transformers.utils import check_min_version\n'

In [2]:
import json
from os.path import join
import jsonlines
def parse(self, response):
    jsonresponse=json.loads(response.text)
    with jsonlines.open('output.jsonl',mode='a') as writer:
        writer.write(jsonresponse)

def get_dataset(file_path):
    dataset = []
    with open(file_path, "r+", encoding="utf8") as f:
        for item in jsonlines.Reader(f):
            dataset.append(item)
    return dataset

def save_dataset(dataset, file_path):
    with open(file_path, "w+", encoding="utf8") as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

train_data = get_dataset(join('data','train.jsonl'))
val_data = get_dataset(join('data','valid.jsonl'))
test_data = get_dataset(join('data','test_public.jsonl'))

In [5]:
from jiayan import PMIEntropyLexiconConstructor
from jiayan import load_lm
from jiayan import CharHMMTokenizer

lm = load_lm(join('jiayan_models','jiayan.klm'))
tokenizer = CharHMMTokenizer(lm)

for data in tqdm(test_data):
    choices = data['choices']
    data['split_choices'] = []
    for choice in choices:
        tokens = list(tokenizer.tokenize(choice))
        data['split_choices'].append(tokens)
save_dataset(test_data, join('data','test_split.jsonl'))
assert 1

100%|██████████| 2720/2720 [00:01<00:00, 2088.89it/s]


In [10]:
import re

train_data = get_dataset(join('data','train.jsonl'))
val_data = get_dataset(join('data','valid.jsonl'))
test_data = get_dataset(join('data','test_public.jsonl'))

def split_verse(verse):
    verse_list = re.split('[，。]', verse)
    entry_list = []
    for verse in verse_list:
        if len(verse) == 5:
            entry_list.append(verse[:2])
            entry_list.append(verse[2:])
        elif len(verse) == 7:
            entry_list.append(verse[:2])
            entry_list.append(verse[2:4])
            entry_list.append(verse[4:])
            entry_list.append(verse[:4])
        else:
            tokens = list(tokenizer.tokenize(verse))
            entry = ''
            for token in tokens:
                if len(token) > 1:
                    entry_list.append(entry)
                    entry = ''
                    entry_list.append(token)
                else:
                    entry += token
            if entry != '':
                entry_list.append(entry)
    return entry_list


for data in tqdm(test_data):
    choices = data['choices']
    data['split_choices'] = []
    for choice in choices:
        tokens = split_verse(choice)
        data['split_choices'].append(tokens)
save_dataset(test_data, join('data','test_split.jsonl'))


100%|██████████| 2720/2720 [00:00<00:00, 47747.52it/s]
